In [7]:
from openpyxl import Workbook
from openpyxl import load_workbook
from tqdm.notebook import tqdm
from os import listdir
import pandas as pd
import tabula
from faker import Factory
from collections import defaultdict

In [6]:
%pip install faker

  Using cached Faker-19.3.1-py3-none-any.whl (1.7 MB)
Note: you may need to restart the kernel to use updated packages.


## Cleaning up Courses Offered

IITD Eacads puts the courses offered up as a very very messy CSV file (they probably export it from the excel themselves) and the excel has a lot of blank merged columns, so it's easier to download the excel file and strip the useless stuff out of it, then export to CSV.

In [18]:
COURSES_OFFERED_RAW_PATH = "./data/courses_offered_raw"
COURSES_OFFERED_CLEAN_PATH = "data/courses_offered"
COURSES_OFFERED_ANON = "data/courses_offered_anon"

In [19]:
datasheets = listdir(COURSES_OFFERED_RAW_PATH)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './data/courses_offered_raw'

In [16]:
def clean_dataframe(df):
    df.columns = df.iloc[0]
    df = df.drop([0])
    df = df.astype({'S.No': 'int32', "Vacancy": "int32", "Current Strength": "int32"})
    df = df.set_index(["S.No"])
    cname_spl = df['Course Name'].str.rsplit('-',1,expand=True)
    (ccode, cname) = (cname_spl[1], cname_spl[0])
    df['Course Name'] = cname
    df['Course Code'] = ccode.str.strip().str.replace(" ","")
    df['Instructor'] = df['Instructor'].str.strip()
    return df

In [15]:
# takes around 10.5 seconds per file, so quite slow
course_dfs = {}
for name in tqdm(datasheets):
    print(name)
    name_no_ext = name.split(".")[0]
    wb = load_workbook(f"{COURSES_OFFERED_RAW_PATH}/{name}")
    ws = wb[wb.sheetnames[0]]
    cols_to_delete = [3,5,7,10,12,13,16,18,21]
    i = 0
    for col in cols_to_delete:
        ws.delete_cols(col-i)
        i += 1
    ws.delete_rows(1,4)
    df = pd.DataFrame(ws.values)
    df = clean_dataframe(df)
    course_dfs[name_no_ext] = df
    #df.to_csv(f"{COURSES_OFFERED_CLEAN_PATH}/{name_no_ext}.csv")

NameError: name 'datasheets' is not defined

## Anonymizing the Data

The Eacademics data is not anonymized wrt professor names, so it's best to go ahead and anonymize before publishing. 

In [14]:
faker = Factory.create('en_IN')
dictnames = defaultdict(faker.simple_profile)

mapping_dict = {}

def change(row):
    email_split = row['Instructor Email'].split("@")
    ename, eaddress = email_split[0], email_split[1]
    email = f"{dictnames[ename]['username']}@{eaddress}"
    name = dictnames[ename]['name']
    row['Instructor Email'] = email
    row['Instructor'] = name.upper()
    mapping_dict[ename] = (name.upper(),email)
    return row

for name in tqdm(course_dfs):
    df = course_dfs[name]
    df_new = df.apply(change, axis=1)
    df_new.to_csv(f"{COURSES_OFFERED_ANON}/{name}.csv")

mapping = []
for key,value in mapping_dict.items():
    mapping.append({"kerberos": key, "name": value[0], "email": value[1]})
    
df_mapping = pd.DataFrame(mapping)
df_mapping.to_csv(f"data/anon_mapping.csv")

NameError: name 'course_dfs' is not defined

## Course Data 

The course structures have to be taken from the Courses of Study; There's no _clean_ way that I could find of doing this, so it's partly manual labour and partly using some utility scripts to generate the credit structure/timetable/DE/PL data.

In [63]:
COS = "Courses-of-Study_2020-2021.pdf"

PROGRAM_4Y_PAGE = {
    "AM1": 44,
    "BB1": 46,
    "CH1": 48,
    "CE1": 53,
    "CS1": 55,
    "EE1": 60,
    "EE3": 62,
    "MS1": 64,
    "ME1": 66,
    "ME2": 68,
    "MT1": 70,
    "PH1": 75,
    "TT1": 77
}
PROGRAM_5Y_PAGE = {
    "CH7": 50,
    "CS5": 57,
    "MT6": 72
}

COLUMNS_LHS = [80, 257, 265, 276, 287]
AREA_LHS = [220, 40, 800, 297]

COLUMNS_RHS = [343, 511, 520, 530, 540]
AREA_RHS = [94, 309, 800, 560]

COLUMNS_CRED_STRUCT = [242]
AREA_CRED_STRUCT_4Y = [106, 40, 220, 295]
AREA_CRED_STRUCT_5Y = [106, 40, 290, 295]

CPLAN_IDX_4Y = [0, 3, 7, 10, 13, 16, 19, 22]
CPLAN_IDX_5Y = [0, 3, 7, 10, 13, 16, 19, 22, 25, 28]
CPLAN_IDX_5Y_SUMMER = [0, 3, 7, 10, 13, 16, 19, 22, 26, 29]

COMMON_COURSES = """
    <eas>
      <course>APL100</course>
      <course>CVL100</course>
      <course>COL100</course>
      <course>ELL101</course>
      <course>MCP100</course>
      <course>MCP101</course>
    </eas>
    <bs>
      <course>CML101</course>
      <course>CMP100</course>
      <course>MTL100</course>
      <course>MTL101</course>
      <course>PYL101</course>
      <course>PYP100</course>
      <course>SBL100</course>
    </bs>
"""

FILE_LOC = "data/course_plans_temp"

In [208]:
for dep in tqdm(PROGRAM_4Y_PAGE):
    strlist = []
    pgno = PROGRAM_4Y_PAGE[dep]
    cred_struct = tabula.read_pdf(COS, area=AREA_CRED_STRUCT_4Y, columns=[242], pages=[pgno])[0]
    cs = {'bs': 24, 'eas': 19, 'huss': 15}
    cs['pl'] = cred_struct.iloc[4][1]
    cs['dc'] = cred_struct.iloc[6][1]
    cs['de'] = cred_struct.iloc[7][1]
    cs['oe'] = cred_struct.iloc[8][1]
    strlist.append(f'<program name="{dep}" type="4y">\n  <credits>\n')
    for key in cs:
        strlist.append(f"    <{key}>{cs[key]:g}</{key}>\n")
    strlist.append('  </credits>\n  <courses>')
    strlist.append(COMMON_COURSES)
    strlist.append('  </courses>\n  <recommended>\n')
    
    plan = tabula.read_pdf(COS, pages=[pgno+1])[0]
    for (i,sem) in zip(range(8),CPLAN_IDX_4Y):
        strlist.append(f'    <sem num="{i+1}">\n')
        for c in plan.iloc[sem][1:10]:
            if not pd.isna(c):
                course = c.replace(" ","")
                strlist.append(f'      <course>{course}</course>\n')
        strlist.append(f'    </sem>\n')
    strlist.append('  </recommended>\n</program>')
        
    with open(f"{FILE_LOC}/{dep}.xml", "w") as f:
        f.write("".join(strlist))

  0%|          | 0/13 [00:00<?, ?it/s]

In [64]:
for dep in tqdm(PROGRAM_5Y_PAGE):
    strlist = []
    pgno = PROGRAM_5Y_PAGE[dep]
    plan = tabula.read_pdf(COS, pages=[pgno+2])[0]
    plist = CPLAN_IDX_5Y if dep == "MT6" else CPLAN_IDX_5Y_SUMMER
    for (i,sem) in zip(range(10),plist):
        strlist.append(f'    <sem num="{i+1}">\n')
        for c in plan.iloc[sem][1:10]:
            if not pd.isna(c):
                course = c.replace(" ","")
                strlist.append(f'      <course>{course}</course>\n')
        strlist.append(f'    </sem>\n')
    
    print("".join(strlist))

  0%|          | 0/3 [00:00<?, ?it/s]

    <sem num="1">
      <course>ELL101</course>
      <course>MCP100</course>
      <course>PYL101</course>
      <course>MTL100</course>
      <course>PYP100</course>
      <course>MCP101</course>
      <course>NIN100</course>
      <course>NEN100</course>
      <course>NLN100</course>
    </sem>
    <sem num="2">
      <course>APL100</course>
      <course>COL100</course>
      <course>CML101</course>
      <course>MTL101</course>
      <course>CMP100</course>
      <course>NEN101</course>
      <course>NLN101</course>
    </sem>
    <sem num="3">
      <course>CLL110</course>
      <course>CLL111</course>
      <course>CLL113</course>
      <course>CML103</course>
      <course>HUL2XX</course>
      <course>CLN101</course>
    </sem>
    <sem num="4">
      <course>CLL121</course>
      <course>CLL122</course>
      <course>CLL231</course>
      <course>CLL251</course>
      <course>SBL100</course>
      <course>APL102</course>
    </sem>
    <sem num="5">
      <course>CLL252</cour

### Scratch

In [61]:
df = tabula.read_pdf(COS, pages=[52])[0]
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Semest,r,Unnamed: 6,Unnamed: 7,L,T,P,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,I,ELL101,MCP100,PYL101,MTL100,PYP100,MCP101,NIN100,NEN100,NLN100,9.0,3.0,14.0,19.0,2.5,31.0
1,NaN,Introduction to Electrical\rEngineering,Introduction to Engineering\rVisualization,Electromagnetic Waves\rand Quantum Mechanics,Calculus,Physics Laboratory,Product Realization through\rManufacturing,Introduction to Engineering\r(Non-graded),Professional Ethics and\r\rSocial Responsibili...,Language and\rWriting Skills-1\r(Non-Graded),NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3125,0042,3104,3104,0042,0042,0021,0010.5,0021,NaN,NaN,NaN,NaN,NaN,NaN
3,II,APL100,COL100,CML101,MTL101,CMP100,NaN,NaN,NEN101,NLN101,12.0,3.0,6.0,18.0,1.5,24.0
4,NaN,Engineering Mechanics,Introduction to Computer\rScience,Introduction to Chemistry,Linear Algebra and Differential\rEquations,Chemistry Laboratory,NaN,NaN,Professional Ethics and\rSocial Responsibility...,Language and\rWriting Skills-2\r(Non-Graded),NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3104,3024,3104,3104,0042,NaN,NaN,0010.5,0021,NaN,NaN,NaN,NaN,NaN,NaN
6,Note: Courses 1-6 above are attended in the gi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,III,CLL110,CLL111,CLL113,CML103,HUL2XX,NaN,CLN101,NaN,NaN,14.0,4.0,2.0,19.0,1.0,22.0
8,NaN,Transport Phenomena,Material and Energy\rBalances,Numerical Methods in\rChemical Engineering,Applied Chemistry: Chemistry\rat Interfaces,NaN,NaN,Introduction to Chemical\rEngineering (Non-gra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,3104,2204,3024,3003,3104,NaN,0021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
cred_struct

,Course Category,Credits
0,Institute Core Courses,NaN
1,Basic Sciences (BS),24.0
2,Engineering Arts and Science (EAS),19.0
3,Humanities and Social Sciences (HuSS),15.0
4,Programme-linked Courses,15.0
5,Departmental Courses,NaN
6,Departmental Core,60.0
7,Departmental Electives,10.0
8,Open Category Courses,10.0
9,Total Graded Credit requirement,153.0


In [202]:
#print(df.dtypes)
for c in df.iloc[22][1:10]:
    if not pd.isna(c):
        print(c)

DE 3
OC2
OC3
HUL3XX


In [218]:
from mimesis import Generic

fake = Generic()
fake_name = fake.person.full_name()
fake_address = fake.address.address()
fake_email = fake.person.email()
fake_phone_number = fake.person.telephone()
print(f'''{fake}
{fake_name}
{fake_email}
{fake_phone_number}''')

Generic <Locale.EN>
Arnold Harvey
renewal1923@live.com
+1-228-049-0532
